In [1]:
from glob import glob
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import periodogram
from scipy.stats import skew, moment, kurtosis
from matplotlib import pyplot as plt
import math
from statsmodels.tsa.stattools import acf 

In [2]:
def parser_apple(aDate, aMicro):
    if not pd.isna(aDate):
        full_date = datetime.fromisoformat(aDate)
        full_date = full_date.replace(microsecond=int(aMicro))
    else :
        full_date = np.NaN
    return full_date

In [3]:
def read_apple(data_path):
    # First check if Apple Data exists in the file
    file = open(data_path, 'r')
    apple_exist = False
    if "Apple" in file.readline():
        #print("Apple Data Found")
        apple_data = pd.read_csv(data_path, usecols=['Apple Time', 'Apple Second Fraction', 'Apple X', 'Apple Y', 'Apple Z', 'Apple Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()
        apple_data = apple_data.rename(columns={apple_data.columns[0]: 'Apple Time'})
        apple_data = apple_data.sort_values(by='Apple Time')
        apple_data['Apple Time'] = apple_data['Apple Time'].apply(lambda x : int(x.timestamp()*1000))
        start = apple_data.iloc[0,0]
        apple_data["Apple Time"] = apple_data['Apple Time'].apply(lambda x: x - start)
        apple_data = apple_data.values
    else:
        #print("No Apple Data")
        apple_data = None
        start = None
    file.close()
    return apple_data, start

In [4]:
def read_garmin(data_path):
    # First check if Garmin Data exists in the file
    file = open(data_path, 'r')
    Garmin_exist = False
    if "Garmin" in file.readline():
        #print("Garmin Data Found")
        garmin_data = pd.read_csv(data_path, usecols=['Garmin Time', 'Garmin X', 'Garmin Y', 'Garmin Z', 'Garmin Magnitude'], parse_dates=[0], infer_datetime_format=True).dropna()
        garmin_data['Garmin Time'] = garmin_data['Garmin Time'].apply(lambda x : int(x.timestamp()*1000))
        start = garmin_data.iloc[0,0]
        garmin_data["Garmin Time"] = garmin_data['Garmin Time'].apply(lambda x: x - start)
        garmin_data = garmin_data.values
    else:
        #print("No Garmin Data")
        garmin_data = None
        start = None
    file.close()
    return garmin_data, start

In [5]:
def read_fitbit(data_path):
    # First check if Fitbit Data exists in the file
    file = open(data_path, 'r')
    fitbit_exist = False
    if "Fitbit" in file.readline():
        #print("Fitbit Data Found")
        fitbit_data = pd.read_csv(data_path, usecols=['Fitbit Time', 'Fitbit Second Fraction', 'Fitbit X', 'Fitbit Y', 'Fitbit Z', 'Fitbit Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()
        if fitbit_data.shape[0] > 0:
            fitbit_data = fitbit_data.rename(columns={fitbit_data.columns[0]: 'Fitbit Time'})
            fitbit_data = fitbit_data.sort_values(by='Fitbit Time')
            fitbit_data['Fitbit Time'] = fitbit_data['Fitbit Time'].apply(lambda x : int(x.timestamp()*1000))
            start = fitbit_data.iloc[0,0]
            fitbit_data["Fitbit Time"] = fitbit_data['Fitbit Time'].apply(lambda x: x - start)
            fitbit_data = fitbit_data.values
        else:
            fitbit_data = None
            start = None
    else:
        #print("No Fitbit Data")
        fitbit_data = None
        start = None
    file.close()
    return fitbit_data, start

In [6]:
def read_actigraph(data_path):
    # First check if Actigraph Data exists in the file
    file = open(data_path, 'r')
    actigraph_exist = False
    if "Actigraph" in file.readline():
        #print("Fitbit Data Found")
        actigraph_data = pd.read_csv(data_path, usecols=['Actigraph Time', 'Actigraph Second Fraction', 'Actigraph X', 'Actigraph Y', 'Actigraph Z', 'Actigraph Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()
        if actigraph_data.shape[0] > 0:
            actigraph_data = actigraph_data.rename(columns={actigraph_data.columns[0]: 'Actigraph Time'})
            actigraph_data = actigraph_data.sort_values(by='Actigraph Time')
            actigraph_data['Actigraph Time'] = actigraph_data['Actigraph Time'].apply(lambda x : int(x.timestamp()*1000))
            start = actigraph_data.iloc[0,0]
            actigraph_data["Actigraph Time"] = actigraph_data['Actigraph Time'].apply(lambda x: x - start)
            actigraph_data = actigraph_data.values
        else:
            actigraph_data = None
            start = None
    else:
        #print("No Actigraph Data")
        actigraph_data = None
        start = None
    file.close()
    return actigraph_data, start
    

In [7]:
def read_actiheart(data_path):
    # First check if Fitbit Data exists in the file
    file = open(data_path, 'r')
    actiheart_exist = False
    if "Actiheart" in file.readline():
        #print("Actiheart Data Found")
        actiheart_data = pd.read_csv(data_path, usecols=['Actiheart ECG Time', 'Actiheart Second Fraction','Actiheart ECG (uV)','Actiheart X', 'Actiheart Y', 'Actiheart Z'], parse_dates=[0], infer_datetime_format=True).dropna()
        actiheart_data['Actiheart ECG Time'] = actiheart_data['Actiheart ECG Time'].apply(lambda x : int(x.timestamp()*1000))
        start = actiheart_data.iloc[0,0]
        actiheart_data["Actiheart ECG Time"] = actiheart_data['Actiheart ECG Time'].apply(lambda x: x - start)
        actiheart_data = actiheart_data.values
    else:
        #print("No Actiheart Data")
        actiheart_data = None
        start = None
    file.close()
    return actiheart_data, start

In [8]:
# This function calculates the dominant acceleration frequency and returns that frequency and it's power
def dom_freq(aWindow):
    # Subtract the mean from signal to get rid of DC offset
    no_dc = aWindow - np.mean(aWindow)
    # Use periodogram to calculate the power of each frequency
    freq, power = periodogram(no_dc, 50) # 50 is the sampling rate of the Apple Watches Accelerometer
    # Select the dominant frequency and power
    max_index = np.argmax(power)
    dom = freq[max_index]
    max_pow = power[max_index]
    return dom, max_pow

In [9]:
def accel_metrics(some_data, start_time, out_path,secagg):
    # Start and stop of daylight savings for 2022
    ds_begin_2022 = datetime(year=2022, month=3, day=13)
    ds_end_2022 = datetime(year=2022, month=11, day=6)
    ds_begin_2023 = datetime(year=2023, month=3, day=12)
    ds_end_2023 = datetime(year=2023, month=11, day=5)
    
    win_size = (secagg+1) * 1000 # Window should be (secagg+1) seconds long. Apple Time is in millisecond
    # Initialize beginning of first window as 0
    win_start = 0
    win_end = (secagg+1)*1000

    # Initialize hash table to store indicies of each second since the sample rate
    # Isn't perfect
    win_start_table = {0: 0}
    cur_sec = 1000
    # Initialize array for output
    output = np.zeros([int(some_data[-1,0]//1000)-secagg, 21])
    output_iter = 0
    for i in range(some_data.shape[0] - 1):
        # CHeck if a second has passed. If so, update win_start_table
        if some_data[i, 0] >= cur_sec:
            # Add the index that the current second begins. Set it's key value
            # To be the corresponding second number.
            win_start_table[cur_sec//1000] = i
            cur_sec += 1000


        if some_data[i+1,0] > win_end :
            # Get 31 second window of data
            rows = win_start_table[win_start]
            x_win= some_data[rows:i+1, 1]
            y_win= some_data[rows:i+1, 2]
            z_win= some_data[rows:i+1, 3]
            mag_win= some_data[rows:i+1, 4]
            # Timestamp output
            output[output_iter, 0] = some_data[win_start_table[int(win_end//1000 - (secagg/2))],0]
            # Caclulate dominant frequency and power
            freq, power = dom_freq(mag_win)
            # Add calc to output
            output[output_iter, 1] = freq
            output[output_iter, 2] = power
            # Calculate MEAN
            output[output_iter, 3] = np.mean(mag_win)
            # Calculate STD
            output[output_iter, 4] = np.std(mag_win)
            # Coefficeint of Variation
            output[output_iter, 5] = output[output_iter, 4] / output[output_iter, 3]
            # Calculate Min
            output[output_iter, 6] = np.min(mag_win)
            # Calculate Max
            output[output_iter, 7] = np.max(mag_win)
            # Calculate 10th Percentile
            output[output_iter, 8] = np.percentile(mag_win, 10)
            # Calculate the 25th Percentile
            output[output_iter, 9] = np.percentile(mag_win, 25)
            # Calculate the 50th Percentile
            output[output_iter, 10] = np.percentile(mag_win, 50)
            # Calculate the 75th Percentile
            output[output_iter, 11] = np.percentile(mag_win, 75)
            # Calculate the 90th Percentile 
            output[output_iter, 12] = np.percentile(mag_win, 90)
            # Calculate the 3rd Moment
            output[output_iter, 13] = moment(mag_win, 3)
            # Calculate the 4th Moment
            output[output_iter, 14] = moment(mag_win, 4)
            # Calculate Skewness
            output[output_iter, 15] = skew(mag_win)
            # Calculate Kurtosis
            output[output_iter, 16] = kurtosis(mag_win)
            # Calculate the autocorrelation with second lag
            output[output_iter, 17] = acf(mag_win, nlags=50)[-1]
            # Correlate X with Y
            output[output_iter, 18] = np.correlate(x_win, y_win)
            # Correlate X with Z
            output[output_iter, 19] = np.correlate(x_win, z_win)
            # Correlate Y with Z
            output[output_iter, 20] = np.correlate(y_win, z_win)
            # Advance window by a second
            win_start += 1
            win_end += 1000
            output_iter += 1

    names = ['Time', 'Dominant Frequency', 'Power', 'Mag Mean', 'Mag STD', 'Mag CoV', 'Mag Min', 'Mag Max', 'Mag 10th Percentile',
               'Mag 25th Percentile', 'Mag 50th Percentile', 'Mag 75th Percentile', 'Mag 90th Percentile', 'Mag 3rd Moment',
               'Mag 4th Moment', 'Mag Skewness', 'Mag Kurtosis', 'Mag 1sec Autocorrelation', 'XY Correlation', 'XZ Correlation', 
               'YZ Correlation']
    out_df = pd.DataFrame(output, columns=names)
    out_df['Time'] = out_df['Time'].apply(lambda x: datetime.fromtimestamp((x+ start_time)//1000))
    if ds_begin_2022 <= out_df.iloc[0,0] <= ds_end_2022 or ds_begin_2023 <= out_df.iloc[0,0] <= ds_end_2023:
        out_df['Time'] = out_df['Time'].apply(lambda x: x + timedelta(hours=4))
    else:
        out_df['Time'] = out_df['Time'].apply(lambda x: x + timedelta(hours=5))
    out_df.to_csv(out_path, index=False)

## RUN BELOW TO CALCULATE APPLE METRICS

In [10]:
pa_dir = "V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/sambit/Desktop/Apple/30_seconds_Magnitude/"

participant_paths = glob(pa_dir + "[0-9][0-9][0-9][0-9]/*aligned.csv")
total_participant = len(participant_paths)
curr_participant = 1
timewindow=30
for participant in participant_paths:
    participant_id = participant[len(pa_dir):len(pa_dir)+4]
    print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
    # Read in data
    apple_accel, start = read_apple(participant)
    if apple_accel is not None:
        # Define output path for metrics
        out_path = out_dir + participant_id + "_accel_metrics.csv"
        # Calculate Metrics and write to file
        print(apple_accel.shape)
        accel_metrics(apple_accel, start, out_path,timewindow)

    else:
        print(f"No Apple Data for {participant_id}")
    
    curr_participant += 1
    


Processing Participant 0118 1/191


C:\Users\sambit\AppData\Local\Temp\ipykernel_8856\4022689757.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  apple_data = pd.read_csv(data_path, usecols=['Apple Time', 'Apple Second Fraction', 'Apple X', 'Apple Y', 'Apple Z', 'Apple Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


(210978, 5)
Processing Participant 0231 2/191


C:\Users\sambit\AppData\Local\Temp\ipykernel_8856\4022689757.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  apple_data = pd.read_csv(data_path, usecols=['Apple Time', 'Apple Second Fraction', 'Apple X', 'Apple Y', 'Apple Z', 'Apple Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


(189871, 5)
Processing Participant 0444 3/191


...

(172428, 5)
Processing Participant 3060 187/191


C:\Users\sambit\AppData\Local\Temp\ipykernel_8856\4022689757.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  apple_data = pd.read_csv(data_path, usecols=['Apple Time', 'Apple Second Fraction', 'Apple X', 'Apple Y', 'Apple Z', 'Apple Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


(190515, 5)
Processing Participant 3063 188/191
No Apple Data for 3063
Processing Participant 3160 189/191


C:\Users\sambit\AppData\Local\Temp\ipykernel_8856\4022689757.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  apple_data = pd.read_csv(data_path, usecols=['Apple Time', 'Apple Second Fraction', 'Apple X', 'Apple Y', 'Apple Z', 'Apple Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


(186912, 5)
Processing Participant 3200 190/191
No Apple Data for 3200
Processing Participant 3201 191/191
No Apple Data for 3201


## RUN BELOW TO CALCULATE GARMIN METRICS

In [12]:
pa_dir = "V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/sambit/Desktop/Garmin/30_seconds_Magnitude/"

participant_paths = glob(pa_dir + "[0-9][0-9][0-9][0-9]/*aligned.csv")
total_participant = len(participant_paths)
curr_participant = 1
timewindow=30
for participant in participant_paths:
    participant_id = participant[len(pa_dir):len(pa_dir)+4]
    if(int(participant_id)>2576):
        try:
            print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
            # Read in data
            garmin_accel, start = read_garmin(participant)
            if garmin_accel is not None:
                # Define output path for metrics
                out_path = out_dir + participant_id + "_accel_metrics.csv"
                # Calculate Metrics and write to file
                print(garmin_accel.shape)
                accel_metrics(garmin_accel, start, out_path,timewindow)

            else:
                print(f"No Garmin Data for {participant_id}")
        except Exception as inst:
                print(inst)
    
    curr_participant += 1

Processing Participant 2584 75/195
(94875, 5)
Processing Participant 2588 76/195
(108250, 5)
Processing Participant 2589 77/195
(107925, 5)
Processing Participant 2596 78/195
(97600, 5)
Processing Participant 2601 79/195
(116075, 5)
Processing Participant 2604 80/195
(100725, 5)
Processing Participant 2605 81/195
(102425, 5)
Processing Participant 2608 82/195
(96675, 5)
Processing Participant 2611 83/195
(118825, 5)
Processing Participant 2612 84/195
(110025, 5)
Processing Participant 2613 85/195
(112150, 5)
Processing Participant 2628 86/195
(107725, 5)
Processing Participant 2629 87/195
(104075, 5)
Processing Participant 2630 88/195
(40475, 5)
Processing Participant 2631 89/195
(109125, 5)
Processing Participant 2635 90/195
(106475, 5)
Processing Participant 2637 91/195
(94950, 5)
Processing Participant 2639 92/195
(124750, 5)
Processing Participant 2640 93/195
(113800, 5)
Processing Participant 2641 94/195
(96625, 5)
Processing Participant 2642 95/195
(89225, 5)
Processing Participa

## RUN BELOW TO CALCULATE FITBIT METRICS

In [8]:
pa_dir = "V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/sambit/Desktop/Fitbit/20_seconds_Magnitude/"

participant_paths = glob(pa_dir + "[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]/*aligned.csv")
total_participant = len(participant_paths)
curr_participant = 1
timewindow=20
for participant in participant_paths:
    participant_id = participant[len(pa_dir):len(pa_dir)+4]
    print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
    # Read in data
    fitbit_accel, start = read_fitbit(participant)
    if fitbit_accel is not None:
        # Define output path for metrics
        out_path = out_dir + participant_id + "_accel_metrics.csv"
        # Calculate Metrics and write to file
        print(fitbit_accel.shape)
        accel_metrics(fitbit_accel, start, out_path,timewindow)

    else:
        print(f"No Fitbit Data for {participant_id}")
    
    curr_participant += 1

Processing Participant 0695 1/1


C:\Users\sambit\AppData\Local\Temp\ipykernel_17816\2210933653.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  fitbit_data = pd.read_csv(data_path, usecols=['Fitbit Time', 'Fitbit Second Fraction', 'Fitbit X', 'Fitbit Y', 'Fitbit Z', 'Fitbit Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


(177747, 5)


### RUN BELOW TO CALCULATE ACTIHEART METRICS

In [30]:
pa_dir = "V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/sambit/Desktop/Actiheart/"

participant_paths = glob(pa_dir + "[0-9][0-9][0-9][0-9]/*aligned.csv")
total_participant = len(participant_paths)
curr_participant = 1

for participant in participant_paths:
    participant_id = participant[len(pa_dir):len(pa_dir)+4]
    print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
    # Read in data
    try:
        actiheart_accel, start = read_actiheart(participant)
        if actiheart_accel is not None:
            # Define output path for metrics
            out_path = out_dir + participant_id + "_accel_metrics.csv"
            # Calculate Metrics and write to file
            print(actiheart_accel.shape)
            accel_metrics(actiheart_accel, start, out_path)

        else:
            print(f"No Actiheart Data for {participant_id}")
    except:pass
    
    curr_participant += 1

Processing Participant 2824 1/7
(199820, 6)
Processing Participant 2987 2/7
Processing Participant 2988 3/7
Processing Participant 2989 4/7
Processing Participant 2995 5/7
Processing Participant 2996 6/7
Processing Participant 2999 7/7


### RUN BELOW TO CALCULATE ACTIGRAPH METRICS

In [12]:
pa_dir = "V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/sambit/Desktop/Actigraph/20_seconds_Magnitude/"

participant_paths = glob(pa_dir + f"[0-9][0-9][0-9][0-9]/*_aligned.csv")
total_participant = len(participant_paths)
curr_participant = 1
timewindow=20
for participant in participant_paths:
    participant_id = participant[len(pa_dir):len(pa_dir)+4]
    print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
    # Read in data
    
    actigraph_accel, start = read_actigraph(participant)
    if actigraph_accel is not None:
        # Define output path for metrics
        out_path = out_dir + participant_id + "_accel_metrics.csv"
        # Calculate Metrics and write to file
        print(actigraph_accel.shape)
        accel_metrics(actigraph_accel, start, out_path,timewindow)

    else:
        print(f"No Actigraph Data for {participant_id}")
    
    curr_participant += 1

Processing Participant 0118 1/191


C:\Users\sambit\AppData\Local\Temp\ipykernel_8812\2134189346.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  actigraph_data = pd.read_csv(data_path, usecols=['Actigraph Time', 'Actigraph Second Fraction', 'Actigraph X', 'Actigraph Y', 'Actigraph Z', 'Actigraph Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


(422000, 5)


C:\Users\sambit\AppData\Local\Temp\ipykernel_8812\2264439755.py:64: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  output[output_iter, 13] = moment(mag_win, 3)
C:\Users\sambit\AppData\Local\Temp\ipykernel_8812\2264439755.py:66: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  output[output_iter, 14] = moment(mag_win, 4)
C:\Users\sambit\AppData\Local\Temp\ipykernel_8812\2264439755.py:68: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  output[output_iter, 15] = skew(mag_win)
C:\Users\sambit\AppData\Local\Temp\ipykernel_8812\2264439755.py:70: RuntimeWarning: Precision loss occurred in moment calculation due to catastroph

Processing Participant 0231 2/191


...

(372000, 5)
Processing Participant 3200 190/191


C:\Users\sambit\AppData\Local\Temp\ipykernel_8812\2134189346.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  actigraph_data = pd.read_csv(data_path, usecols=['Actigraph Time', 'Actigraph Second Fraction', 'Actigraph X', 'Actigraph Y', 'Actigraph Z', 'Actigraph Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


(392000, 5)
Processing Participant 3201 191/191


C:\Users\sambit\AppData\Local\Temp\ipykernel_8812\2134189346.py:7: FutureWarning: 
        Use pd.to_datetime instead.

  actigraph_data = pd.read_csv(data_path, usecols=['Actigraph Time', 'Actigraph Second Fraction', 'Actigraph X', 'Actigraph Y', 'Actigraph Z', 'Actigraph Magnitude'], parse_dates=[[0,1]], date_parser=parser_apple).dropna()


(392000, 5)


### Combine Agg and Actigraph Data

In [3]:
agg_dir="C:/Users/sambit/Desktop/Actigraph/Agg_Metrics/"
combined_dir = "C:/Users/sambit/Desktop/Actigraph/30_seconds_Magnitude/"
out_dir="C:/Users/sambit/Desktop/Actigraph/"
participant_paths = glob(agg_dir + "[0-9][0-9][0-9][0-9]_agg_metrics.csv")
total_participant = len(participant_paths)
curr_participant = 1

for participant in participant_paths:
    participant_id = participant[-20:-16]
    print(f"Processing Participant {participant_id} {curr_participant}/{total_participant}")
    agg_path=agg_dir+f"{participant_id}_agg_metrics.csv"
    
    combined_path=combined_dir+f"{participant_id}_accel_metrics.csv"
    if os.path.isfile(agg_file) and os.path.isfile(combined_file):
        try:
            agg_df = pd.read_csv(agg_path, parse_dates=['Time'], infer_datetime_format=True)
        except:
            agg_df = pd.read_csv(agg_path, parse_dates=['time'], infer_datetime_format=True)
            
        try:
            # Update column name in hr_df to 'Time'
            agg_df.rename(columns={'time': 'Time'}, inplace=True)
        except:pass
        
        
        
        out_path=out_dir +f"Combined_Actiheart_Metrics_With_Agg_30_seconds/{participant_id}_combined_metrics.csv"
        try:
            agg_df.merge(hr_df, left_on='Time', right_on='Time', how='left').to_csv(out_path, index=False)
        
        
            
    curr_participant += 1

'0118'

In [2]:
agg_dir="V:/ACOI/R01 - W4K/3_PA protocol/1_Participants/"
out_dir = "C:/Users/sambit/Desktop/Actigraph/Agg_Metrics/"

ds_begin = datetime(year=2022, month=3, day=13)
ds_end = datetime(year=2023, month=7, day=6)
participant_paths = glob(agg_dir + "[0-9][0-9][0-9][0-9]/")
total_participant = len(participant_paths)
curr_participant = 1
column_values=[]

for participant in participant_paths:
    p_id=participant[-5:-1]
    print(f"Agg file {p_id} {curr_participant}/{total_participant}")
    try:
        agg_file=participant+f"{p_id}_agg.csv"
        agg_file=pd.read_csv(agg_file)
    except:pass
    try:
        agg_file = agg_file.iloc[:, [0, 1, 2, 7,8,8,10,11,12,13,14,15,16,17,18,19,54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,65,66]]
        agg_file.to_csv(out_dir+f"{p_id}_agg_metrics.csv")
    except:
        pass
    
    curr_participant+=1

Agg file 0118 1/192
Agg file 0231 2/192
Agg file 0444 3/192
Agg file 0601 4/192
Agg file 0618 5/192
Agg file 0632 6/192
Agg file 0695 7/192
Agg file 0819 8/192
Agg file 0967 9/192
Agg file 1333 10/192
Agg file 1760 11/192
Agg file 1772 12/192
Agg file 1779 13/192
Agg file 1799 14/192
Agg file 1854 15/192
Agg file 1974 16/192
Agg file 1991 17/192
Agg file 2007 18/192
Agg file 2031 19/192
Agg file 2069 20/192
Agg file 2113 21/192
Agg file 2186 22/192
Agg file 2187 23/192
Agg file 2271 24/192
Agg file 2272 25/192
Agg file 2321 26/192
Agg file 2355 27/192
Agg file 2433 28/192
Agg file 2453 29/192
Agg file 2454 30/192
Agg file 2455 31/192
Agg file 2456 32/192
Agg file 2457 33/192
Agg file 2458 34/192
Agg file 2470 35/192
Agg file 2471 36/192
Agg file 2473 37/192
Agg file 2474 38/192
Agg file 2475 39/192
Agg file 2476 40/192
Agg file 2479 41/192
Agg file 2480 42/192
Agg file 2495 43/192
Agg file 2496 44/192
Agg file 2497 45/192
Agg file 2498 46/192
Agg file 2499 47/192
Agg file 2500 48/192
A

C:\Users\sambit\AppData\Local\Temp\ipykernel_9024\3759735326.py:16: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  agg_file=pd.read_csv(agg_file)


Agg file 2678 123/192
Agg file 2679 124/192
Agg file 2680 125/192
Agg file 2681 126/192
Agg file 2683 127/192
Agg file 2684 128/192
Agg file 2685 129/192
Agg file 2686 130/192


C:\Users\sambit\AppData\Local\Temp\ipykernel_9024\3759735326.py:16: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  agg_file=pd.read_csv(agg_file)


Agg file 2687 131/192
Agg file 2688 132/192
Agg file 2689 133/192
Agg file 2690 134/192
Agg file 2691 135/192
Agg file 2692 136/192
Agg file 2706 137/192
Agg file 2709 138/192
Agg file 2710 139/192
Agg file 2711 140/192
Agg file 2712 141/192
Agg file 2753 142/192
Agg file 2774 143/192
Agg file 2775 144/192
Agg file 2776 145/192
Agg file 2778 146/192
Agg file 2779 147/192
Agg file 2780 148/192
Agg file 2781 149/192
Agg file 2782 150/192
Agg file 2783 151/192
Agg file 2788 152/192
Agg file 2789 153/192
Agg file 2790 154/192
Agg file 2792 155/192
Agg file 2793 156/192
Agg file 2794 157/192
Agg file 2795 158/192
Agg file 2796 159/192
Agg file 2797 160/192
Agg file 2798 161/192
Agg file 2801 162/192
Agg file 2812 163/192
Agg file 2813 164/192
Agg file 2814 165/192


...

Agg file 2815 166/192
Agg file 2816 167/192
Agg file 2817 168/192
Agg file 2818 169/192
Agg file 2819 170/192
Agg file 2820 171/192
Agg file 2821 172/192
Agg file 2822 173/192
Agg file 2823 174/192
Agg file 2824 175/192


C:\Users\sambit\AppData\Local\Temp\ipykernel_9024\3759735326.py:16: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  agg_file=pd.read_csv(agg_file)


Agg file 2987 176/192
Agg file 2988 177/192
Agg file 2989 178/192
Agg file 2990 179/192
Agg file 2991 180/192
Agg file 2993 181/192
Agg file 2995 182/192
Agg file 2996 183/192
Agg file 2999 184/192
Agg file 3057 185/192
Agg file 3058 186/192
Agg file 3059 187/192
Agg file 3060 188/192


C:\Users\sambit\AppData\Local\Temp\ipykernel_9024\3759735326.py:16: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  agg_file=pd.read_csv(agg_file)


Agg file 3063 189/192
Agg file 3160 190/192
Agg file 3200 191/192
Agg file 3201 192/192
